# Guided Project: Predicting House Sale Prices Using Linear Regression in Python

In this project I will be working with Linear Regression in Python to predict the sale prices of houses in Ames, Iowa from 2006 to 2010. The data documentation can be found [here](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt).

## Importing Data

In [28]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold

#Set pandas to be able to see all columns
pd.options.display.max_columns = 999

data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
data.shape

(2930, 82)

In [29]:
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


## The first model 
This is a simple model where Gr Liv Area (above ground living area) is the only feature and SalePrice is the target. 

In [30]:
def transform_features(df):
    return df 

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    train_numeric = train.select_dtypes(include=['float', 'integer'])
    test_numeric = test.select_dtypes(include=['float', 'integer'])
    
    features = train_numeric.columns.drop('SalePrice')
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse 

In [31]:
transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data)

rmse

57088.25161263909

## Feature Engineering
To improve on the simple model, I will transform, remove, or create new features that better captures information using already existing features. 

* Handling missing values:
    * All columns:
        * Drop any with 5% or more missing values for now.
    * Text columns:
        * Drop any with 1 or more missing values for now.
    * Numerical columns:
        * For columns with missing values, fill in with the most common value in that column

In [32]:
data.isnull().sum()[data.isnull().sum() > 0]

Lot Frontage       490
Alley             2732
Mas Vnr Type        23
Mas Vnr Area        23
Bsmt Qual           80
Bsmt Cond           80
Bsmt Exposure       83
BsmtFin Type 1      80
BsmtFin SF 1         1
BsmtFin Type 2      81
BsmtFin SF 2         1
Bsmt Unf SF          1
Total Bsmt SF        1
Electrical           1
Bsmt Full Bath       2
Bsmt Half Bath       2
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual        159
Garage Cond        159
Pool QC           2917
Fence             2358
Misc Feature      2824
dtype: int64

In [33]:
#drop all columns with missing values greater than 5% of total rows 
nullvaluecount = data.isnull().sum()
missingfivepercent = nullvaluecount[nullvaluecount > (len(data)/20)].index
data = data.drop(missingfivepercent, axis=1)


In [34]:
data.isnull().sum()[data.isnull().sum() > 0]

Mas Vnr Type      23
Mas Vnr Area      23
Bsmt Qual         80
Bsmt Cond         80
Bsmt Exposure     83
BsmtFin Type 1    80
BsmtFin SF 1       1
BsmtFin Type 2    81
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Electrical         1
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Cars        1
Garage Area        1
dtype: int64

In [35]:
#drop all text columns with more than one missing values
textcolumns = data.select_dtypes(include='object')
nulltextcolumnscount = textcolumns.isnull().sum()
missingtext = nulltextcolumnscount[nulltextcolumnscount >= 1].index
data = data.drop(missingtext, axis=1)


In [36]:
data.isnull().sum()[data.isnull().sum() > 0]

Mas Vnr Area      23
BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Cars        1
Garage Area        1
dtype: int64

In [37]:
#checking the modes of numeric columns with missing values 
for x in data.isnull().sum()[data.isnull().sum() > 0].index:
    print (x)
    print (data[x].mode()[0])

Mas Vnr Area
0.0
BsmtFin SF 1
0.0
BsmtFin SF 2
0.0
Bsmt Unf SF
0.0
Total Bsmt SF
0.0
Bsmt Full Bath
0.0
Bsmt Half Bath
0.0
Garage Cars
2.0
Garage Area
0.0


In [38]:
#fill remaining null numeric values with mode of column
for col in data.isnull().sum()[data.isnull().sum() > 0].index:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [39]:
#checking to make sure there are no missing values
data.isnull().sum().value_counts()


0    64
dtype: int64

## New features
Instead of having just Year Remod/Add, we can subtract it from the year built column to get a better idea of time between when the house was built and remodeled.

In [40]:
#create a new feature that better captures how long from the year built that the house was remodeled
years_until_remod = data['Year Remod/Add'] - data['Year Built']
years_until_remod[years_until_remod < 0]

850   -1
dtype: int64

In [41]:
#adding features to DataFrame
data['Years Until Remod'] = years_until_remod 

In [42]:
#removing rows with negative values for the two new features
neg_rows = data[(data['Years Until Remod'] < 0)].index
data = data.drop(neg_rows, axis=0) 

## Removing Features
I removed Year Remod/Add since it is now redundant with the new feature I created. I also dropped PID and Order since they are not useful for my purposes. Mo Sold, Yr Sold, Sale Type, and Sale Condition were also dropped due to information leakage these columns can cause since they would only exist after a prediction. 

In [43]:
#drop Year Built and Year Remod/Add since we created better features and no longer require these columns
data = data.drop(["Year Remod/Add"], axis = 1)

#drop columns that are not useful for ML(PID & ORDER)
data = data.drop(['PID', 'Order'], axis = 1)

#drop columns that leak data about sale(Mo Sold, Yr Sold, Sale Type, Sale Condition)
data = data.drop(['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition'], axis=1)

## The Updated Model
I updated the function transform_features to include the changes I made in the cells above.

In [44]:
def transform_features(df):
    nullvaluecount = df.isnull().sum()
    missingfivepercent = nullvaluecount[nullvaluecount > (len(df)/20)].index
    df = df.drop(missingfivepercent, axis=1)
    
    textcolumns = df.select_dtypes(include='object')
    nulltextcolumnscount = textcolumns.isnull().sum()
    missingtext = nulltextcolumnscount[nulltextcolumnscount >= 1].index
    df = df.drop(missingtext, axis=1)

    for col in df.isnull().sum()[df.isnull().sum() > 0].index:
        df[col].fillna(df[col].mode()[0], inplace=True)

    years_until_remod = df['Year Remod/Add'] - df['Year Built']
    years_until_remod[years_until_remod < 0]

    df['Years Until Remod'] = years_until_remod 

    neg_rows = df[(df['Years Until Remod'] < 0)].index
    df = df.drop(neg_rows, axis=0) 

    df = df.drop(["Year Remod/Add"], axis = 1)
    df = df.drop(['PID', 'Order'], axis = 1)
    df = df.drop(['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition'], axis=1)
    return df 

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    train_numeric = train.select_dtypes(include=['float', 'integer'])
    test_numeric = test.select_dtypes(include=['float', 'integer'])
    
    features = train_numeric.columns.drop('SalePrice')
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse 

data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data)

rmse

57100.480580407326

## Feature Selection
Feature selection is a very critial part in any model. I will select features based on their correlation to SalePrice. Although here I will arbitrarily select 0.4, it is something to experiment with.

In [45]:
#finding which features are strongly correlated to SalePrice
numeric_data = transform_data.select_dtypes(include=['float', 'integer'])
corrmat = numeric_data.corr()
sorted_corr = corrmat['SalePrice'].abs().sort_values()
strong_corr = sorted_corr[sorted_corr > 0.4]
transform_data = transform_data.drop(sorted_corr[sorted_corr < 0.4].index, axis=1)
strong_corr

BsmtFin SF 1     0.433221
Fireplaces       0.474722
TotRms AbvGrd    0.495514
Mas Vnr Area     0.502223
Full Bath        0.545594
Year Built       0.558435
1st Flr SF       0.621671
Total Bsmt SF    0.632536
Garage Area      0.640126
Garage Cars      0.647801
Gr Liv Area      0.706801
Overall Qual     0.799268
SalePrice        1.000000
Name: SalePrice, dtype: float64

## Nominal columns
Some of the nominal columns would be useful to change into categorical columns then dummy variables and included in the model. Somethings to keep an eye on is the amount of unique values each columns has, due to fact that for each unique value, there will be a new column when turned into a dummy variable. Also if there is a categorical column where one unique value applies to the majority of the data, then there is no variability for the model to capture.

In [46]:
#create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

In [47]:
#check which nominal features are still in transform_data
transform_cat_cols = []
for col in nominal_features:
    if col in transform_data.columns:
        transform_cat_cols.append(col)
transform_cat_cols


['MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [48]:
#checking for unique values from catagorical columns if there are too many it could add a lot of columns for each dummy variable, too few then there is no variability in the data for the model to capture 
uniqueness_counts = transform_data[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
uniqueness_counts

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

Here I chose a unique count of 10 as the cutoff for no real reason and this is a value to experiment with.

In [49]:
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_data = transform_data.drop(drop_nonuniq_cols, axis=1)

In [50]:
#convert objects to catergorical dtype 
text_cols = transform_data.select_dtypes(include='object')
for col in text_cols:
    transform_data[col] = transform_data[col].astype('category')

#convert catergorical columns to dummy variables and remove the original column
transform_data = pd.concat([transform_data, pd.get_dummies(transform_data.select_dtypes(include=['category']))], axis=1)
transform_data.head()

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Year Built,Roof Style,Roof Matl,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,BsmtFin SF 1,Total Bsmt SF,Heating,Heating QC,Central Air,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,SalePrice,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Grvl,Street_Pave,Lot Shape_IR1,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Utilities_AllPub,Utilities_NoSeWa,Utilities_NoSewr,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Land Slope_Gtl,Land Slope_Mod,Land Slope_Sev,Condition 1_Artery,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Artery,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNn,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Roof Style_Flat,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_ClyTile,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exter Qual_Ex,Exter Qual_Fa,Exter Qual_Gd,Exter Qual_TA,Exter Cond_Ex,Exter Cond_Fa,Exter Cond_Gd,Exter Cond_Po,Exter Cond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating QC_Ex,Heating QC_Fa,Heating QC_Gd,Heating QC_Po,Heating QC_TA,Central Air_N,Central Air_Y,Kitchen Qual_Ex,Kitchen Qual_Fa,Kitchen Qual_Gd,Kitchen Qual_Po,Kitchen Qual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Paved Drive_N,Paved Drive_P,Paved Drive_Y
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,1Story,6,1960,Hip,CompShg,112.0,TA,TA,CBlock,639.0,1080.0,GasA,Fa,Y,1656,1656,1,TA,7,Typ,2,2.0,528.0,P,215000,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,Feedr,Norm,1Fam,1Story,5,1961,Gable,CompShg,0.0,TA,TA,CBlock,468.0,882.0,GasA,TA,Y,896,896,1,TA,5,Typ,0,1.0,730.0,Y,105000,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,1Story,6,1958,Hip,CompShg,108.0,TA,TA,CBlock,923.0,1329.0,GasA,TA,Y,1329,1329,1,Gd,6,Typ,0,1.0,312.0,Y,172000,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,1Story,7,1968,Hip,CompShg,0.0,Gd,TA,CBlock,1065.0,2110.0,GasA,Ex,Y,2110,2110,2,Ex,8,Typ,2,2.0,522.0,Y,244000,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,2Story,5,1997,Gable,CompShg,0.0,TA,TA,PConc,791.0,928.0,GasA,Gd,Y,928,1629,2,TA,6,Typ,1,2.0,482.0,Y,189900,0,0,0,0,0,1,0,0,1,1,0,0

## The Updated Model v2
I updated the function select_features to include the changes I made in the cells above.

In [51]:
def transform_features(df):
    nullvaluecount = df.isnull().sum()
    missingfivepercent = nullvaluecount[nullvaluecount > (len(df)/20)].index
    df = df.drop(missingfivepercent, axis=1)
    
    textcolumns = df.select_dtypes(include='object')
    nulltextcolumnscount = textcolumns.isnull().sum()
    missingtext = nulltextcolumnscount[nulltextcolumnscount >= 1].index
    df = df.drop(missingtext, axis=1)

    for col in df.isnull().sum()[df.isnull().sum() > 0].index:
        df[col].fillna(df[col].mode()[0], inplace=True)

    years_until_remod = df['Year Remod/Add'] - df['Year Built']
    years_until_remod[years_until_remod < 0]

    df['Years Until Remod'] = years_until_remod 

    neg_rows = df[(df['Years Until Remod'] < 0)].index
    df = df.drop(neg_rows, axis=0) 

    df = df.drop(["Year Remod/Add"], axis = 1)
    df = df.drop(['PID', 'Order'], axis = 1)
    df = df.drop(['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition'], axis=1)
    return df  

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numeric_data = df.select_dtypes(include=['float', 'integer'])
    corrmat = numeric_data.corr()
    sorted_corr = corrmat['SalePrice'].abs().sort_values()
    strong_corr = sorted_corr[sorted_corr > coeff_threshold]
    df = df.drop(sorted_corr[sorted_corr < coeff_threshold].index, axis=1)

    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", "Misc Feature", "Sale Type", "Sale Condition"]

    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > uniq_threshold].index
    df = df.drop(drop_nonuniq_cols, axis=1)


    text_cols = df.select_dtypes(include='object')
    cat_cols = df.select_dtypes(include='category')
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    return df 

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    train_numeric = train.select_dtypes(include=['float', 'integer'])
    test_numeric = test.select_dtypes(include=['float', 'integer'])
    
    features = train_numeric.columns.drop('SalePrice')
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse 

data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data)

rmse

38070.24691032001

## The Updated Model v3
I updated the function train_and_test to include the ability of cross validation. If k=0, a hold validation will be used. If k=1, a simple cross validation is used and if k>1 then k-fold cross validation is used.  

In [52]:
def transform_features(df):
    nullvaluecount = df.isnull().sum()
    missingfivepercent = nullvaluecount[nullvaluecount > (len(df)/20)].index
    df = df.drop(missingfivepercent, axis=1)
    
    textcolumns = df.select_dtypes(include='object')
    nulltextcolumnscount = textcolumns.isnull().sum()
    missingtext = nulltextcolumnscount[nulltextcolumnscount >= 1].index
    df = df.drop(missingtext, axis=1)

    for col in df.isnull().sum()[df.isnull().sum() > 0].index:
        df[col].fillna(df[col].mode()[0], inplace=True)

    years_until_remod = df['Year Remod/Add'] - df['Year Built']
    years_until_remod[years_until_remod < 0]

    df['Years Until Remod'] = years_until_remod 

    neg_rows = df[(df['Years Until Remod'] < 0)].index
    df = df.drop(neg_rows, axis=0) 

    df = df.drop(["Year Remod/Add"], axis = 1)
    df = df.drop(['PID', 'Order'], axis = 1)
    df = df.drop(['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition'], axis=1)
    return df  

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numeric_data = df.select_dtypes(include=['float', 'integer'])
    corrmat = numeric_data.corr()
    sorted_corr = corrmat['SalePrice'].abs().sort_values()
    strong_corr = sorted_corr[sorted_corr > coeff_threshold]
    df = df.drop(sorted_corr[sorted_corr < coeff_threshold].index, axis=1)

    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", "Misc Feature", "Sale Type", "Sale Condition"]

    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > uniq_threshold].index
    df = df.drop(drop_nonuniq_cols, axis=1)


    text_cols = df.select_dtypes(include='object')
    cat_cols = df.select_dtypes(include='category')
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    return df 

def train_and_test(df, k=0):
    if k == 0:
        train = df[0:1460]
        test = df[1460:]

        train_numeric = train.select_dtypes(include=['float', 'integer'])
        test_numeric = test.select_dtypes(include=['float', 'integer'])

        features = train_numeric.columns.drop('SalePrice')
        lr = LinearRegression()
        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], predictions)
        rmse = np.sqrt(mse)
    
        return rmse 
    
    else: 
        numeric_data = df.select_dtypes(include=['integer', 'float'])
        features = numeric_data.columns.drop("SalePrice")
    
        kf = KFold(k, shuffle=True, random_state=1)
        lr = LinearRegression()
        mses = cross_val_score(lr, X=df[features], y=df['SalePrice'], scoring='neg_mean_squared_error', cv=kf)
        absmse = [abs(x) for x in mses]
        rootmse = [x**0.5 for x in absmse]
        avg_rmse = np.mean(a=rootmse)
        print (rootmse)
        return avg_rmse
    
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data, 5)

rmse

[37373.75129136145, 34835.90326558279, 30555.79355632141, 26318.839291510496, 30852.775211757973]


31987.412523306826

In [53]:
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
transform_data = transform_features(data)
filtered_data = select_features(transform_data, 0.4, 30)
rmse = train_and_test(filtered_data, 5)

rmse

[35462.63272993677, 32240.999234981504, 29292.161758727936, 24858.343821391743, 28675.43221395922]


30105.91395179943